<a href="https://colab.research.google.com/github/Mohamed2bdelaziz/Feature-selection-with-Particle-Swarm-Optimization-PSO-/blob/main/Feature_selection_with_Particle_Swarm_Optimization_(PSO).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Swarm Optimization Algorithm: An Overview.


> Swarm Optimization Algorithm is a type of computational algorithm inspired by the collective behavior of decentralized, self-organized systems found in nature, such as bird flocks, fish schools, and insect colonies. The most common forms of swarm optimization algorithms include `Particle Swarm Optimization (PSO)` and Ant Colony Optimization (ACO).

### Particle Swarm Optimization (PSO):

> 1. <u>Inspiration:</u> Derived from the social behavior of `birds flocking` or `fish schooling`.
2. <u>Mechanism:</u> Uses a population of candidate solutions called `particles`. Each particle moves through the solution space influenced by `its own best-known position` and the `best-known positions of other particles` in the swarm.
3. <u>Objective:</u> Find the optimal solution by **iteratively improving candidate solutions** with regard to a given measure of quality or fitness.

### How It Works: Particle Swarm Optimization (PSO)

> 1. <u>Initialization:</u> Initialize a population of particles with **random positions** and **velocities** in the solution space.
2. <u>Evaluation:</u> Evaluate the fitness of each particle according to the objective function.
3. <u>Update Velocities and Positions:</u>
Each particle updates its velocity based on three factors:
  * *Inertia:* The particle's previous velocity.
  * *Personal Best:* The distance to the particle's best-known position.
  * *Global Best:* The distance to the swarm's best-known position.<br>
Update the particle’s position by **adding its new velocity to its current position**.
4. <u>Iteration:</u> Repeat the evaluation and update steps until a termination criterion is met (e.g., a maximum number of iterations or a satisfactory fitness level).

### Particle Swarm Optimization (PSO) for feature selection:

Particle Swarm Optimization (PSO) can be adapted for feature selection in a tabular dataset. Feature selection aims to choose a subset of relevant features for model building, which can improve the performance and interpretability of a machine learning model.

## Imports.

In [16]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

## Loading COVID-19 dataset from kaggle.

In [2]:
# load the data in a ziped file using kaggle api command
! kaggle datasets download -d meirnizri/covid19-dataset

Dataset URL: https://www.kaggle.com/datasets/meirnizri/covid19-dataset
License(s): CC0-1.0
  0% 0.00/4.66M [00:00<?, ?B/s]
100% 4.66M/4.66M [00:00<00:00, 100MB/s]


In [3]:
# unziping loaded zip file
! unzip /content/covid19-dataset.zip

Archive:  /content/covid19-dataset.zip
  inflating: Covid Data.csv          


In [4]:
# read data from the csv file using pandas
data_path = '/content/Covid Data.csv' # here's the data path
covid_df = pd.read_csv(data_path) # load the data into a DataFrame

# showing random 8 samples from the data
covid_df.sample(8)

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
231857,2,4,2,1,9999-99-99,97,2,28,97,2,...,2,2,2,2,2,2,2,2,7,97
145293,2,4,1,1,9999-99-99,97,2,46,2,2,...,2,2,2,2,2,2,2,2,3,97
896465,1,12,1,1,9999-99-99,97,2,30,2,2,...,2,2,2,2,2,2,2,2,7,97
750805,2,12,1,1,9999-99-99,97,2,25,2,1,...,2,2,2,2,2,2,2,2,7,97
159170,2,4,1,1,9999-99-99,97,2,47,2,2,...,2,2,1,2,2,2,2,2,5,97
613383,1,12,1,1,9999-99-99,97,2,17,2,2,...,2,2,2,2,2,2,2,2,3,97
342069,1,6,2,2,07/05/2020,2,1,47,97,2,...,2,2,1,2,2,2,2,2,6,2
1038241,2,12,2,1,9999-99-99,97,2,26,97,2,...,2,2,2,2,2,2,2,2,7,97


## Processing

In [5]:
covid_df.CLASIFFICATION_FINAL.value_counts()

CLASIFFICATION_FINAL
7    499250
3    381527
6    128133
5     26091
1      8601
4      3122
2      1851
Name: count, dtype: int64

In [6]:
"""
As the range of "CLASIFFICATION_FINAL" column varies away,
while the pationt is in covid-19 when his classification value below 4,
here is the "binarize_diagnosis" function that maps the column into binary values
where 1 for covid cariers and 0 for not cariers
"""
def binarize_diagnosis(
    classification_value : int
    ) -> int:
    return int(classification_value < 4)

In [7]:
# apply the "binarize_diagnosis" function over the "CLASIFFICATION_FINAL" column
covid_df.CLASIFFICATION_FINAL = covid_df.CLASIFFICATION_FINAL.apply(binarize_diagnosis)

covid_df.CLASIFFICATION_FINAL.value_counts()

CLASIFFICATION_FINAL
0    656596
1    391979
Name: count, dtype: int64

In [17]:
X = covid_df.drop(columns = ['CLASIFFICATION_FINAL', 'DATE_DIED']).values
X = StandardScaler().fit_transform(X)

y = covid_df.CLASIFFICATION_FINAL.values#.reshape(-1, 1)

print(f"Data shape: {X.shape}, While target's shape: {y.shape}")

Data shape: (1048575, 19), While target's shape: (1048575,)


## Feature selection (PSO).

1. Define the Problem and Objective Function
>The objective function could be the performance metric of a machine learning model (e.g., accuracy, F1-score) using a subset of features.

In [18]:
# Objective function
def objective_function(
    selected_features : list[bool],
    X = X, y = y
    ) -> float:
    if np.sum(selected_features) == 0:
        return 0  # Avoid empty feature subset
    X_selected = X[:, selected_features == 1]
    clf = LogisticRegression(random_state=42)
    clf.fit(X_selected, y)
    y_pred = clf.predict(X_selected)
    return accuracy_score(y, y_pred)

2. Encode Particles
> Each particle represents a subset of features. This can be encoded as a binary vector where 1 means the feature is selected and 0 means it is not.

3. Initialize Particles
> Initialize particle positions (binary vectors) and velocities (continuous values).

4. Evaluate Fitness
> Evaluate the fitness of each particle by training and validating a machine learning model using the selected features.

In [25]:
# Parameters
num_particles = 6
dimensions = X.shape[1]
max_iter = 10
c1 = 2.0
c2 = 2.0
w = 0.7
w_decay = 0.99

# Initialize particle positions and velocities
particles_position = np.random.randint(2, size=(num_particles, dimensions))
particles_velocity = np.random.uniform(low=-1, high=1, size=(num_particles, dimensions))
pBest_position = np.copy(particles_position)
pBest_value = np.array([objective_function(p) for p in tqdm(particles_position)])
gBest_position = pBest_position[np.argmax(pBest_value)]
gBest_value = np.max(pBest_value)



100%|██████████| 6/6 [00:19<00:00,  3.21s/it]


In [26]:
print(f"Best selected features vector: {particles_position[np.argmax(pBest_value)]}")
print(f"Accuracy of best selected features vector: {gBest_value :0.3f}")

Best selected features vector: [0 1 1 1 1 1 1 0 0 0 0 1 1 0 0 0 1 0 1]
Accuracy of best selected features vector: 0.662


5. Update Velocities and Positions
> Update velocities and positions using PSO equations. Use a sigmoid function to convert continuous velocities into binary positions.

6. Update Personal and Global Bests
> Update the personal best positions and the global best position based on the fitness evaluations.

7. Iterate
> Repeat the update steps until the stopping criterion is met.

8. Return the Best Feature Subset
> Return the best feature subset found during the iterations.

In [28]:
# PSO main loop
for t in range(max_iter):
    for i in tqdm(range(num_particles)):
        # Update velocities
        r1 = np.random.rand(dimensions)
        r2 = np.random.rand(dimensions)
        particles_velocity[i] = (
            w * particles_velocity[i] +
            c1 * r1 * (pBest_position[i] - particles_position[i]) +
            c2 * r2 * (gBest_position - particles_position[i])
        )

        # Update positions using sigmoid function
        sigmoid = 1 / (1 + np.exp(-particles_velocity[i]))
        particles_position[i] = np.where(np.random.rand(dimensions) < sigmoid, 1, 0)

        # Evaluate fitness
        fitness = objective_function(particles_position[i])

        # Update personal best
        if fitness > pBest_value[i]:
            pBest_position[i] = particles_position[i]
            pBest_value[i] = fitness

    # Update global best
    if np.max(pBest_value) > gBest_value:
        gBest_position = pBest_position[np.argmax(pBest_value)]
        gBest_value = np.max(pBest_value)

    # Decay inertia weight
    w *= w_decay

    # Print progress
    print(f"Iteration {t+1}/{max_iter}, Global Best Value: {gBest_value:0.3f}")



100%|██████████| 6/6 [00:26<00:00,  4.43s/it]


Iteration 1/10, Global Best Value: 0.662



100%|██████████| 6/6 [00:34<00:00,  5.68s/it]


Iteration 2/10, Global Best Value: 0.662



100%|██████████| 6/6 [00:31<00:00,  5.29s/it]


...

Iteration 8/10, Global Best Value: 0.662



100%|██████████| 6/6 [00:21<00:00,  3.55s/it]


Iteration 9/10, Global Best Value: 0.662



100%|██████████| 6/6 [00:25<00:00,  4.22s/it]

Iteration 10/10, Global Best Value: 0.662


In [29]:
# Results
print("Best feature subset:", gBest_position)
print("Best accuracy:", gBest_value)

Best feature subset: [0 1 1 1 1 1 1 0 0 1 0 0 1 1 0 1 0 0 1]
Best accuracy: 0.6615091910449896


In [38]:
selected_cols = covid_df.drop(columns = ['CLASIFFICATION_FINAL', 'DATE_DIED']).columns[np.array(gBest_position).astype(bool)]

print("Selected columns are:", *selected_cols.values, sep="\n")

Selected columns are:
MEDICAL_UNIT
SEX
PATIENT_TYPE
INTUBED
PNEUMONIA
AGE
COPD
HIPERTENSION
OTHER_DISEASE
OBESITY
ICU
